In [1]:
import configparser
import os

import chromadb
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import Chroma
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.docstore.document import Document
from langchain_openai import OpenAI
from langchain.chains.query_constructor.base import load_query_constructor_chain
from langchain.chains import RetrievalQA, combine_documents

import pandas as pd 



In [3]:
probes_df = pd.read_csv('../data/Catalog - Probes.csv')
probes_df.fillna("", inplace=True)
probes_df.drop(columns=['Description'], inplace=True)
probes_df['Compatible_Systems'] = probes_df['Compatible_Systems'].apply(lambda x: [system.strip() for system in x.split(',')])
type(probes_df['Compatible_Systems'])



pandas.core.series.Series

: 

In [19]:
def get_openai_key():
    config = configparser.ConfigParser()
    config.read('config.ini')
    api_key = config['DEFAULT']['OpenAI_API_Key']
    return api_key

def initialize_openai_embeddings():
    # Load the OpenAI embeddings
    api_key = get_openai_key()
    openai_embeddings = OpenAIEmbeddings(api_key=api_key, model="text-embedding-3-large")

    # Define a custom Embeddings class
    class CustomEmbeddings(Embeddings):
        openai_embeddings: OpenAIEmbeddings  # Define the openai_embeddings field

        def __init__(self, openai_embeddings):
            self.openai_embeddings = openai_embeddings

        def embed_documents(self, texts):
            return self.openai_embeddings.embed_documents(texts)

        def embed_query(self, text):
            return self.openai_embeddings.embed_query(text)

    # Create an instance of the custom Embeddings class
    embeddings = CustomEmbeddings(openai_embeddings)

    return embeddings

In [20]:
probes_df = pd.read_csv('../data/Catalog - Probes.csv')
probes_df.fillna("", inplace=True)

# Create a list of Document objects
documents = []
for index, row in probes_df.iterrows():
    # Ensure page_content is a string, replace NaN with an empty string
    page_content = str(row['Description']) if pd.notna(row['Description']) else ""
    
    metadata = {
        "Manufacturer": row['Manufacturer'],
        "Probe_Model": row['Probe_Model'],
        "Connection_Type": row['Connection_Type'],
        "Compatible_Systems": str(row['Compatible_Systems'].split(', ')),
        "Array_Type": row['Array_Type'],
        "Frequency_Range": row['Frequency_Range'],
        "Applications": str(row['Applications'].split(', ')), 
        "Stock": row['Stock']
    }
    
    document = Document(page_content=page_content, metadata=metadata)
    documents.append(document)

In [21]:
# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=get_openai_key())

# Create the Chroma vector store
vectorstore = Chroma.from_documents(documents, embeddings)

In [44]:
metadata_field_info = [
    AttributeInfo(name="Manufacturer", description="The company or brand that manufactured the ultrasound probe.", type="string"),
    AttributeInfo(name="Probe_Model", description="The model name of the specific ultrasound probe.", type="string"),
    AttributeInfo(name="Connection_Type", description="The type of connector or port used to connect the probe to the ultrasound system and differentiate probes with the same model name and manufacturer.", type="string"),
    AttributeInfo(name="Compatible_Systems", description="Ultrasound systems that work with the specific probe model. These ultrasound machines are made by the same manufacturer of the probes they are compatible with.", type="string"),
    AttributeInfo(name="Array_Type", description="The array type of the probe which refers to the configuration and arrangement of crystals within the probe that affect how the ultrasound beam is shaped, steered, and focused.", type="string"),
    AttributeInfo(name="Frequency_Range", description="The range of frequencies (measured in MHz) at which the probe can operate to produce ultrasound waves. Higher frequencies provide higher resolution images and are ideal for visualizing small, superficial structures, however, they cannot visualize deep tissues effectively. Lower frequencies provide deeper penetration to reach deeper tissues and structures.", type="string"),
    AttributeInfo(name="Stock", description="The number of units of the specific probe model that are currently in stock or available for sale.", type="string")
]

# Initialize the LLM
llm = OpenAI(temperature=.2, 
             openai_api_key=get_openai_key(),
             model="gpt-3.5-turbo-instruct"
             )

document_content_description = "A summary of the ultrasound probe/transducer that contains information about the maker/manufacturer, model, compatible systems, array type, frequency ranges, and application."

search_kwargs = {
    'top_k': 5,  # Limit the number of results to the top 5
    'threshold': 0.8  # Only consider results with a similarity score above 0.8
}

sq_retriever = SelfQueryRetriever.from_llm(
    llm=llm, 
    vectorstore=vectorstore, 
    document_contents=document_content_description, 
    metadata_field_info=metadata_field_info,
    verbose=True
)



In [50]:
def ask_chatbot(query):
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=sq_retriever,
        return_source_documents=False,
    )

    result = qa.invoke(query)
    print(f"Query:\n {query}\nResponse:\n {result['result']}\n")

ask_chatbot("What are the compatible systems for the ATL C5-2 probe?")

ask_chatbot("What systems does the ATL C5-2 probe work with?")

ask_chatbot("What systems are the ATL C5-2 probe compatible with?")



Query:
 What are the compatible systems for the ATL C5-2 probe?
Response:
  The compatible systems for the ATL C5-2 probe are the HDI 1500, HDI 3000, HDI 3500, and HDI 5000.

Query:
 What systems does the ATL C5-2 probe work with?
Response:
  The ATL C5-2 probe works with ultrasound systems.

The ATL C5-2 probe is a medical device used for ultrasound imaging. It is a type of transducer, which is a device that converts one form of energy into another. In this case, the probe converts electrical energy into sound waves that are used to create images of the body.

Answer: The ATL C5-2 probe works with ultrasound systems.

Query:
 What systems are the ATL C5-2 probe compatible with?
Response:
  The ATL C5-2 probe is compatible with ATL ultrasound systems.

I don't know.



In [49]:
ask_chatbot("What are the compatible systems for the ATL C5-2 probe?")

ask_chatbot("What systems does the ATL C5-2 probe work with?")

Query:
 What are the compatible systems for the ATL C5-2 probe?

Response:
  The compatible systems for the ATL C5-2 probe are the HDI 1500, HDI 3000, HDI 3500, and HDI 5000.
Query:
 What systems does the ATL C5-2 probe work with?

Response:
  The ATL C5-2 probe works with ATL ultrasound systems.


I don't know.


In [31]:
ask_chatbot("What type of probe is the C5-2?")

Query:
 What type of probe is the C5-2?

Response:
  The C5-2 is a convex array probe.


In [15]:
ask_chatbot("What company makes the C5-2 probe?")

Query:
 What company makes the C5-2 probe?

Response:
  Philips


In [28]:
ask_chatbot("What models are compatible with iE 33?")

Query:
 What models are compatible with iE 33?

Response:
  The Philips S4-2 and the Philips L15-7io are both compatible with the iE33 ultrasound system.


In [30]:
# False query
ask_chatbot("Which linear probes are compatible with iE 33?")

Query:
 Which linear probes are compatible with iE 33?

Response:
 

I don't know.


In [26]:
ask_chatbot("What probe models are compatible with LOGIQ 700?")

Query:
 What probe models are compatible with LOGIQ 700?

Response:
 
The LOGIQ 700 is compatible with a variety of probe models, including linear, convex, phased array, and endocavity probes.


In [37]:
ask_chatbot("Do you have any C5-1 probes in stock?")



Query:
 Do you have any C5-1 probes in stock?

Response:
  I don't know.


In [38]:
ask_chatbot("How many C5-1 probes in stock?")

Query:
 How many C5-1 probes in stock?

Response:
  I do not have enough information to answer this question. The context provided does not mention the number of C5-1 probes in stock.


In [45]:
ask_chatbot("How many number of units of C5-1 probes in stock or available for sale?") 

Query:
 How many number of units of C5-1 probes in stock or available for sale?

Response:
  I do not have access to information about the current stock or availability of C5-1 probes. This information would need to be obtained from a Philips representative or authorized distributor.


In [43]:
ask_chatbot("Who makes the C5-1 probe?")
ask_chatbot("Who makes the C5-1 transducer?")
ask_chatbot("Who makes the ATL C5-1 probe?")


Query:
 Who makes the C5-1 probe?

Response:
  Philips
Query:
 Who makes the C5-1 transducer?

Response:
  The Philips C5-1 transducer is made by Philips.
Query:
 Who makes the ATL C5-1 probe?

Response:
  The manufacturer of the ATL C5-1 probe is ATL Ultrasound.

I don't know.
